In [6]:
#= 📦 Importy i załadowanie danych =#
using JLD2, Printf, Statistics, LinearAlgebra
include("src/AD.jl")
include("src/NN.jl")

file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [7]:
#= 🔄 Przygotowanie danych =#
if size(X_train, 1) < size(X_train, 2)
    X_train = X_train
else
    X_train = X_train'
end
if size(X_test, 1) < size(X_test, 2)
    X_test = X_test
else
    X_test = X_test'
end
X_train_f32 = Float32.(X_train)
X_test_f32 = Float32.(X_test)
dataset = NN.DataLoader((X_train_f32, y_train), batchsize=32, shuffle=true);

In [8]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 16, NN.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dropout(0.3),
    NN.Dense(256, 64, NN.relu),
    NN.Dropout(0.3),
    NN.Dense(64, 1, NN.sigmoid)
);

In [9]:
#= 🎯 Metryka dokładności =#
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [10]:
#= 🚀 Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 4
NN.train_model(model, dataset, X_test_f32, y_test, opt, epochs)

Epoch: 1 (266.17s) 	Train: (l: 0.6091, a: 0.6775) 	Test: (l: 0.5238, a: 0.7537)
Epoch: 2 (238.05s) 	Train: (l: 0.4994, a: 0.7671) 	Test: (l: 0.4785, a: 0.7758)
Epoch: 3 (277.44s) 	Train: (l: 0.4677, a: 0.7883) 	Test: (l: 0.4570, a: 0.7948)
Epoch: 4 (246.83s) 	Train: (l: 0.4427, a: 0.8018) 	Test: (l: 0.4333, a: 0.8079)
